<a href="https://colab.research.google.com/github/WildAlex37/g_search_lite/blob/main/G_search_lite_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import os
import ipywidgets as widgets
from IPython.display import display
pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)


from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
df = pd.read_parquet('df_igdb.parquet')

In [3]:
df.columns

Index(['id', 'name', 'metacritic', 'released', 'website', 'rating', 'playtime',
       'ratings_count', 'suggestions_count', 'game_series_count',
       'reviews_count', 'platforms', 'developers', 'genres', 'esrb_rating',
       'year', 'decennie', 'Card', 'RPG', 'Shooter', 'Strategy', 'Action',
       'Sports', 'Massively Multiplayer', 'Platformer', 'Puzzle',
       'Educational', 'Family', 'Casual', 'Indie', 'Simulation', 'Arcade',
       'Fighting', 'Racing', 'Board Games', 'Adventure', '', 'PlayStation 5',
       'Commodore / Amiga', 'SEGA Saturn', 'SNES', 'Dreamcast',
       'Nintendo Switch', 'Atari 5200', 'Xbox Series S/X', '3DO', 'Atari 7800',
       'SEGA CD', 'Game Boy Advance', 'Nintendo 64', 'Xbox', 'Atari Lynx',
       'Nintendo DS', 'Xbox 360', 'Atari Flashback', 'Neo Geo', 'Apple II',
       'Nintendo 3DS', 'NES', 'PS Vita', 'Game Boy', 'Linux', 'iOS', 'Genesis',
       'PC', 'Jaguar', 'SEGA 32X', 'Atari ST', 'Wii U', 'Web', 'PlayStation 3',
       'Game Boy Color', 'Gam

In [4]:
print(df.isna().sum())

id                            0
name                          0
metacritic               344574
released                      0
website                  292717
rating                        0
playtime                      0
ratings_count                 0
suggestions_count             0
game_series_count             0
reviews_count                 0
platforms                     0
developers                 2301
genres                        0
esrb_rating              298929
year                          0
decennie                      0
Card                          0
RPG                           0
Shooter                       0
Strategy                      0
Action                        0
Sports                        0
Massively Multiplayer         0
Platformer                    0
Puzzle                        0
Educational                   0
Family                        0
Casual                        0
Indie                         0
Simulation                    0
Arcade  

In [5]:
df.head(3)

,id,name,metacritic,released,website,rating,playtime,ratings_count,suggestions_count,game_series_count,reviews_count,platforms,developers,genres,esrb_rating,year,decennie,Card,RPG,Shooter,Strategy,Action,Sports,Massively Multiplayer,Platformer,Puzzle,Educational,Family,Casual,Indie,Simulation,Arcade,Fighting,Racing,Board Games,Adventure,,PlayStation 5,Commodore / Amiga,SEGA Saturn,SNES,Dreamcast,Nintendo Switch,Atari 5200,Xbox Series S/X,3DO,Atari 7800,SEGA CD,Game Boy Advance,Nintendo 64,Xbox,Atari Lynx,Nintendo DS,Xbox 360,Atari Flashback,Neo Geo,Apple II,Nintendo 3DS,NES,PS Vita,Game Boy,Linux,iOS,Genesis,PC,Jaguar,SEGA 32X,Atari ST,Wii U,Web,PlayStation 3,Game Boy Color,Game Gear,Classic Macintosh,PSP,PlayStation 2,macOS,Atari 8-bit,PlayStation 4,Android,Wii,Atari XEGS,PlayStation,SEGA Master System,GameCube,Xbox One,Atari 2600,Nintendo DSi
0,3498,Grand Theft Auto V,97.0,2013-09-17,http://www.rockstargames.com/V/,4.48,69.0,4289.0,426.0,9.0,4334.0,PC||Xbox Series S/X||PlayStation 5||PlayStatio...,Rockstar North,Action,Mature,2013.0,2010.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,3328,The Witcher 3: Wild Hunt,92.0,2015-05-18,https://thewitcher.com/en/witcher3,4.67,50.0,3939.0,688.0,6.0,3996.0,PC||Xbox One||Nintendo Switch||PlayStation 4,CD PROJEKT RED,Action||Adventure||RPG,Mature,2015.0,2010.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,4200,Portal 2,95.0,2011-04-18,http://www.thinkwithportals.com/,4.61,11.0,3613.0,589.0,2.0,3645.0,Xbox One||PlayStation 3||PC||Xbox 360||Linux||...,Valve Software,Shooter||Puzzle,Everyone 10+,2011.0,2010.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0


In [6]:
# Traiter les valeurs manquantes
df_cleaned = df.drop(columns=['metacritic', 'esrb_rating', 'website','platforms','genres'])

# Extraire uniquement les colonnes des genres et des plateformes (exclure les colonnes non numériques)
genre_platform_columns = [col for col in df_cleaned.columns if df_cleaned[col].dtype in ['int64', 'float64']]
X = df_cleaned[genre_platform_columns]
# Définir les caractéristiques (X) et la cible (y)
df_cleaned = df_cleaned.drop(['id','released','developers',''], axis=1)

df_sample = df_cleaned.sample(n=5000, random_state=42)

X = df_sample.drop(['name'], axis=1)  # Exclure les colonnes non pertinentes
# Cible (par exemple une colonne contenant les labels)
y = df_sample['name']


In [7]:
X.info()
print(y.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 241672 to 42431
Data columns (total 78 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   rating                 5000 non-null   float64
 1   playtime               5000 non-null   float64
 2   ratings_count          5000 non-null   float64
 3   suggestions_count      5000 non-null   float64
 4   game_series_count      5000 non-null   float64
 5   reviews_count          5000 non-null   float64
 6   year                   5000 non-null   float64
 7   decennie               5000 non-null   float64
 8   Card                   5000 non-null   int64  
 9   RPG                    5000 non-null   int64  
 10  Shooter                5000 non-null   int64  
 11  Strategy               5000 non-null   int64  
 12  Action                 5000 non-null   int64  
 13  Sports                 5000 non-null   int64  
 14  Massively Multiplayer  5000 non-null   int64  
 15  Pla

In [21]:
# Filtrer les classes avec peu d'exemples
class_counts = y.value_counts()
min_class_size = 50
classes_to_keep = class_counts[class_counts >= min_class_size].index
filtered_df = df_sample[df_sample['name'].isin(classes_to_keep)]
print('Répartition des classes après filtrage :')
print(y.value_counts())

Répartition des classes après filtrage :
name
Reigning Cats                                                                                       1
Drop (philcheese)                                                                                   1
Turret Defense (you will need to download scratch to play it)                                       1
Snowflakery                                                                                         1
Superhero Rescue City                                                                               1
Steam Marines 2                                                                                     1
Apostle 3D DEMO 0.20                                                                                1
Genkai Tokki: Castle Panzers                                                                        1
TURNABOUT (itch)                                                                                    1
Tribal Defense                      

In [19]:


# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir les modèles
models = {
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier()
}

# Fonction pour évaluer les modèles avec StratifiedKFold
def evaluate_models_cv(models, X, y, n_splits=2):
    results = {}
    skf = StratifiedKFold(n_splits=2)
    for name, model in models.items():
        scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
        results[name] = np.mean(scores)
    return results

# Évaluer les modèles
results = evaluate_models_cv(models, X, y)

# Afficher les résultats
for model_name, accuracy in results.items():
    print(f'{model_name} Average Accuracy: {accuracy:.4f}')

ValueError: n_splits=2 cannot be greater than the number of members in each class.

In [32]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# 1. Saisie de l'utilisateur
jeu_saisi = input("Saisissez le nom d'un jeu : ")

# 2. Recherche du jeu dans le DataFrame
if jeu_saisi in df_cleaned['name'].values:
    jeu_selectionne = df_cleaned[df_cleaned['name'] == jeu_saisi]

    # 3. Trouver le genre et les plateformes du jeu sélectionné
    genre_columns = ['Card', 'RPG', 'Shooter', 'Strategy', 'Action', 'Sports',
                     'Massively Multiplayer', 'Platformer', 'Puzzle', 'Educational',
                     'Family', 'Casual', 'Indie', 'Simulation', 'Arcade', 'Fighting',
                     'Racing', 'Board Games', 'Adventure']

    platform_columns = ['PlayStation 5', 'Commodore / Amiga', 'SEGA Saturn', 'SNES', 'Dreamcast',
                        'Nintendo Switch', 'Atari 5200', 'Xbox Series S/X', '3DO', 'Atari 7800',
                        'SEGA CD', 'Game Boy Advance', 'Nintendo 64', 'Xbox', 'Atari Lynx',
                        'Nintendo DS', 'Xbox 360', 'Atari Flashback', 'Neo Geo', 'Apple II',
                        'Nintendo 3DS', 'NES', 'PS Vita', 'Game Boy', 'Linux', 'iOS', 'Genesis',
                        'PC', 'Jaguar', 'SEGA 32X', 'Atari ST', 'Wii U', 'Web', 'PlayStation 3',
                        'Game Boy Color', 'Game Gear', 'Classic Macintosh', 'PSP', 'PlayStation 2',
                        'macOS', 'Atari 8-bit', 'PlayStation 4', 'Android', 'Wii', 'Atari XEGS',
                        'PlayStation', 'SEGA Master System', 'GameCube', 'Xbox One', 'Atari 2600',
                        'Nintendo DSi']

    genre_selectionne = jeu_selectionne[genre_columns].values.flatten()
    platform_selectionne = jeu_selectionne[platform_columns].values.flatten()
    genre_du_jeu = genre_columns[np.where(genre_selectionne == 1)[0][0]]  # Trouver le genre du jeu sélectionné
    platform_du_jeu = platform_columns[np.where(platform_selectionne == 1)[0][0]]  # Trouver la plateforme du jeu sélectionné

    # 4. Filtrer le DataFrame pour ne conserver que les jeux du même genre et plateforme
    df_filtered = df_cleaned[(df_cleaned[genre_du_jeu] == 1) & (df_cleaned[platform_du_jeu] == 1)]

    # 5. Vérifier s'il y a assez de jeux dans le genre et plateforme sélectionnés
    if len(df_filtered) <= 1:
        print(f"Pas assez de jeux dans le genre '{genre_du_jeu}' et sur la plateforme '{platform_du_jeu}' pour faire des recommandations.")
    else:
        # 6. Appliquer la pondération aux colonnes des genres et des plateformes pour les jeux filtrés
        df_filtered_weighted_genres = df_filtered[genre_columns].copy()
        df_filtered_weighted_platforms = df_filtered[platform_columns].copy()

        # 7. Concaténer les colonnes des genres et des plateformes
        df_filtered_weighted = pd.concat([df_filtered_weighted_genres, df_filtered_weighted_platforms], axis=1)

        # 8. Extraire les caractéristiques du jeu sélectionné
        X_jeu = df_filtered_weighted.loc[df_filtered['name'] == jeu_saisi].values.flatten()

        # 9. Préparer les données pour le modèle KNN
        X_all = df_filtered_weighted.drop('name', axis=1, errors='ignore').values  # Les caractéristiques sans la colonne 'name'
        y_all = df_filtered['name']  # Les étiquettes des jeux

        # Séparer les données en ensembles d'entraînement et de test
        X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

        # Entraîner le modèle KNN
        knn = KNeighborsClassifier(n_neighbors=5)  # Vous pouvez ajuster n_neighbors selon vos besoins
        knn.fit(X_train, y_train)

        # Calculer la précision du modèle
        accuracy = knn.score(X_test, y_test)

        # 10. Trouver les jeux les plus similaires
        distances, indices = knn.kneighbors([X_jeu], n_neighbors=6)  # Inclut le jeu sélectionné lui-même

        # 11. Afficher les jeux les plus similaires
        top_n = 5  # Par exemple, recommander les 5 jeux les plus similaires
        similar_games_indices = indices.flatten()[1:]  # Exclure le jeu lui-même
        jeux_similaires = df_filtered.iloc[similar_games_indices]['name']

        print(f"Jeux recommandés similaires à '{jeu_saisi}' dans le genre '{genre_du_jeu}' et sur la plateforme '{platform_du_jeu}':")
        for jeu in jeux_similaires:
            print(jeu)

        print(f"\nAccuracy du modèle KNN: {accuracy:.4f}")
else:
    print(f"Le jeu '{jeu_saisi}' n'existe pas dans la base de données.")


Saisissez le nom d'un jeu : Portal 2
Jeux recommandés similaires à 'Portal 2' dans le genre 'Shooter' et sur la plateforme 'Xbox 360':
INVERSION
Mass Effect 3: Earth
Toy Soldiers: Cold War
Cabelas Dangerous Hunts 2009
Red Faction: Battlegrounds

Accuracy du modèle KNN: 0.0000
